In [6]:
!conda install -c dglteam dgl-cuda10.1
!pip install dgl-cu101

/bin/bash: conda: command not found
     |████████████████████████████████| 36.2MB 79kB/s 


In [7]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [8]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories: ', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories:  7


In [9]:
g = dataset[0]

In [10]:
print('Node Features: ',g.ndata)
print("Edge Features: ", g.edata)

Node Features:  {'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge Features:  {}


In [23]:
g.ndata['label'].shape

torch.Size([2708])

In [39]:
from dgl.nn import GraphConv

class GCN(nn.Module):
  def __init__(self, in_feats, h_feats, num_classes):
    super(GCN, self).__init__()
    self.conv1 = GraphConv(in_feats, h_feats)
    self.conv2 = GraphConv(h_feats, num_classes)

  def forward(self, g, in_feat):
    h = F.relu(self.conv1(g, in_feat))
    h = self.conv2(g, h)
    return h

In [40]:
# create the model
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [54]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']


    for e in range(150):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = criterion(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [55]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.158 (best 0.158), test acc: 0.125 (best 0.125)
In epoch 5, loss: 1.886, val acc: 0.430 (best 0.444), test acc: 0.424 (best 0.438)
In epoch 10, loss: 1.801, val acc: 0.468 (best 0.468), test acc: 0.462 (best 0.462)
In epoch 15, loss: 1.692, val acc: 0.588 (best 0.588), test acc: 0.589 (best 0.589)
In epoch 20, loss: 1.560, val acc: 0.658 (best 0.658), test acc: 0.674 (best 0.674)
In epoch 25, loss: 1.409, val acc: 0.682 (best 0.682), test acc: 0.700 (best 0.700)
In epoch 30, loss: 1.243, val acc: 0.714 (best 0.714), test acc: 0.732 (best 0.732)
In epoch 35, loss: 1.069, val acc: 0.730 (best 0.730), test acc: 0.747 (best 0.744)
In epoch 40, loss: 0.897, val acc: 0.754 (best 0.754), test acc: 0.760 (best 0.760)
In epoch 45, loss: 0.737, val acc: 0.768 (best 0.768), test acc: 0.768 (best 0.768)
In epoch 50, loss: 0.595, val acc: 0.772 (best 0.772), test acc: 0.768 (best 0.770)
In epoch 55, loss: 0.476, val acc: 0.778 (best 0.778), test acc: 0.767 (best 0